# file path

In [ ]:
INSERTS:str = "./../../testymolo/media/old_sql/VIRUS_1-2004-11-04_Inserts_2.sql"
TABLES: str = "./../../testymolo/media/old_sql/VIRUS_1-2004-11-04_Tables.sql"

tables_csv: str = "./../../testymolo/media/tables_csv/"


# tables and inserts

In [ ]:
### caution when not all rows same nbr of col
def custom_csv_parser_to_list(key:str):
    infilepath: str = os.path.join(tables_csv, key+'.csv')
    with open(infilepath ,'r') as handle:

        rows:list = []

        for line in handle.readlines():
            line = line.strip(';') # removing terminal ';' 
            values:list = [] # re-initialize value list
            val:str = "" # current parsing value
            inquote:bool = False # re-initialize in quote : False
            quote_char:str = '' # ' or "
            is_string:bool = False
            for c in line:
                if(c == ','): # encounter comma
                    if(not inquote): ## and not in quote
                        val = val.strip("'")
                        val = val.strip('"')
                        if(len(val) > 0):
                            if(not is_string and val != "NULL" and val != "None"):
                                values.append(int(val))
                            else:
                                values.append(val)
                            is_string = False
                        else:
                            values.append(None)
                        val = ""
                        continue
                elif(c == '"' or c == "'"): # encounter quote
                    if(inquote): # already in quote 
                        if(quote_char == c):  # encounter ending quote mark
                            inquote = False
                        else: # encounter the other quote mark 
                            pass
                    else: # encounter starting quote mark
                        inquote = True
                        is_string = True
                        quote_char = c
                elif((c == ' ' or c == '\n') and not inquote):  # get rid of whitespaces
                    continue
                val += c
            if(len(val)>0):
                val = val.strip("'")
                val = val.strip('"')
                if(not is_string and val != "NULL" and val != "None"):
                    values.append(int(val))
                else:
                    values.append(val)
                is_string = False

            if(len(values) > 0):
                rows.append(values)
                #print(values)  
        
        #print("nbr of col", set([len(row) for row in rows]))
        #print("nbr of rows", len(rows))
        return rows

import os

data_keys:list = [ "Biblio_Struct", "CAZy_DB", "CAZy_GB_GP", "CAZy_PDB", "CAZy_PP", "CAZy_SP", "CAZyModO",
    "Cz_EC", "EC_num", "Fam_biblio", "ModO_Composition", "ModO_CrossRefs", "ModO_Families", "ModO_Limits",
    "Motifs", "Organism", "Prot_biblio", "Prot_Infos", "Prot_MOTIF", "Prot_MUT", "Prot_REG", "Prot_RI"]

data:dict = { key:custom_csv_parser_to_list(key) for key in data_keys }


In [ ]:
### sort tables
def sort_tab(tab:list, primary_key_indexes:tuple=(0,1)):
    import operator
    new_tab:list = [   ]

    for j in range(len(tab)):
        for k in primary_key_indexes:
            tab[j][k] = int(tab[j][k])

    for i in reversed(primary_key_indexes):
        new_tab = sorted(tab, key=operator.itemgetter(i))
    return new_tab

"""
for tab in [ "CAZy_DB", "CAZy_GB_GP", "CAZy_PDB", "CAZy_SP",
    "Motifs", "Organism", "Cz_EC",]:
    data[tab] = sort_tab(data[tab], (0,))

for tab in ["CAZy_PP", "ModO_Composition", "ModO_Limits"]:
    data[tab] = sort_tab(data[tab], (0,1))

for tab in ["Prot_Infos", "Prot_MOTIF", "Prot_MUT", "Prot_REG", "Prot_RI"]:
    data[tab] = sort_tab(data[tab], (0,2))
"""

In [ ]:
### write corrected tables

import csv

def write_table(key):
    infilepath: str = os.path.join(tables_csv, key+'.csv')
    with open(infilepath, 'w') as filehandler:
        for row in data[key]:
            line:str = ""
            for val in row:
                if(isinstance(val, str)):
                    line += "'"+val+"'"
                else:
                    line += str(val)
                line += ", "
            filehandler.write(line[:-2]+'\n')

#for tab in data:
#    write_table(tab)

## Exploration of results

In [ ]:
## no assotiated : 609/2620
## multiple assotiated : 428/2620

import csv

no_associated_annotations:list = []
multiple_associated_annotations:list = []

with open("./Seq_annotation_list") as filehandler:
    csvcontent = csv.reader(filehandler)
    for row in csvcontent:
        if(row[-1] == "False"):
            no_associated_annotations.append(row)
        elif(len(row[-1].split(' ')) > 1):
            multiple_associated_annotations.append(row)

print(len(no_associated_annotations))
print(len(multiple_associated_annotations))


In [ ]:
count:dict = {}
for row in no_associated_annotations:
    if(row[1] in count):
        count[row[1]] += 1
    else:
        count[row[1]] = 1

for item in count:
    print(f"{item}:{count[item]}/{len(data[item])}")

## unfinished annotations per families

In [ ]:
uncomplete_fams:list = []
with open("./Modulo_list") as filehandler:
    csvcontent = csv.reader(filehandler)
    for row in csvcontent:
        if(row[-1] == "False"):
            if(row[0] not in uncomplete_fams):
                uncomplete_fams.append(row[0])
print(uncomplete_fams)

In [ ]:
import re
import json

from os import path

from Bio import SeqIO

testydata: str = "./../../testymolo/media/data/"
multifasta: str = path.join(testydata,"sequences.fasta")



def Get_fastaseq_from_file(id:str):
    # Read the Sequences fasta file
    result = None
    with open(multifasta) as handle:
        for record in SeqIO.parse(handle,format="fasta"):
            if(record.id == id):
                return str(record.seq)


In [ ]:
Get_fastaseq_from_file("473")

In [3]:

import re
import json

from os import path

from Bio import SeqIO, Seq


tables_csv: str = "./../../testymolo/media/tables_csv/"


testydata: str = "./../../testymolo/media/data/"
data_json: str = path.join(testydata,"data.json")
multifasta: str = path.join(testydata,"sequences.fasta")


def custom_csv_parser_to_list(infilepath:str) -> list:
    with open(path.join(tables_csv, infilepath+".csv") ,'r') as handle:

        rows:list = []

        for line in handle.readlines():
            line = line.strip(';') # removing terminal ';' 
            values:list = [] # re-initialize value list
            val:str = "" # current parsing value
            inquote:bool = False # re-initialize in quote : False
            quote_char:str = '' # ' or "
            for c in line:
                if(c == ','): # encounter comma
                    if(not inquote): ## and not in quote
                        val = val.strip("'")
                        val = val.strip('"')
                        if(len(val) > 0):
                            values.append(val)
                        else:
                            values.append(None)
                        val = ""
                        continue
                elif(c == '"' or c == "'"): # encounter quote
                    if(inquote): # already in quote 
                        if(quote_char == c):  # encounter ending quote mark
                            inquote = False
                        else: # encounter the other quote mark 
                            pass
                    else: # encounter starting quote mark
                        inquote = True
                        quote_char = c
                elif((c == ' ' or c == '\n') and not inquote):  # get rid of whitespaces
                    continue
                val += c
            if(len(val)>0):
                val = val.strip("'")
                val = val.strip('"')
                values.append(val)

            if(len(values) > 0):
                rows.append(values)
                #print(values)  
        
        print("nbr of col", set([len(row) for row in rows]))
        print("nbr of rows", len(rows))
        return rows


def Get_fastaseq_from_file(id:str) -> Seq:
    # Read the Sequences fasta file
    with open(multifasta) as handle:
        for record in SeqIO.parse(handle,format="fasta"):
            if(record.id == id):
                return record.seq


def Get_name_from_CAZy_DB(id:str) -> str:
    CAZy_DB = custom_csv_parser_to_list("CAZy_DB")
    for item in CAZy_DB:
        if(item[0] == id) :
            return item[1]


def Get_org_name_from_CAZy_DB(id:str) -> str:
    CAZy_DB = custom_csv_parser_to_list("CAZy_DB")
    for item in CAZy_DB:
        if(item[0] == id) :
            return item[3]


def Get_limits_of_subseq(id:str) -> tuple:
    Compositions = custom_csv_parser_to_list("ModO_Composition")
    compo = Compositions[int(id)]
    Limits = custom_csv_parser_to_list("ModO_Limits")
    for lim in Limits:
        if(lim[0] == compo[0] and lim[1] == compo[1]):
            return (lim[-2], lim[-1])



def parse_data():
    log = open("./log", "w")

    # Read the DATA json file
    data:dict = {}
    with open(data_json) as handle:
        data = json.load(handle)


    # Parse data dict
    ## species -> Organism__Tax_id
    for species in data:
        Tax_id:str = species

        ## sequence -> CAZy_DB__DB_ac
        for sequence in data[species]:
            ### data_ac -> CAZy_DB__DB_ac
            data_ac:str = sequence
            ### get fasta seq of CAZy_DB__DB_ac
            fastaseq:str = Get_fastaseq_from_file(sequence)
            ### get protein name
            name:str = Get_name_from_CAZy_DB(sequence)
            ### get organism name
            org_name:str = Get_org_name_from_CAZy_DB(sequence)
            ### create header 
            header:str = name.replace(" ", "_")+"__"+"(@"+data_ac+")"

            ### organism doesnt exist yet 
            #if(len(Organism.objects.filter(id=int(Tax_id)))<1): 
                ### add organism in MODELS (tax_id, name, phylogeny?)
            log.write("""
            Organism_model = Organism.objects.create(
                id = int(Tax_id),
                name = org_name,
                phylogeny = ""
            )\n""")
            #else:
            #    Organism_model = Organism.objects.get(id=int(Tax_id))

            ### add protein in MODELS (Tax_id, data_ac, fastaseq, name, header)
            log.write("""
            Protein_model = Protein.objects.create(
                organism = Organism_model,
                name = name,
                data_ac = int(data_ac),
                header = header,
                sequence = fastaseq
            )\n""")

            for subseq in data[species][sequence]["subseq"]:
                ### key   -> composition_lineNumber
                ### value -> domain__id
                start, end = Get_limits_of_subseq(sequence)

                ### add subseq in MODELS (origin, profile?, start, end)
                log.write("""
                Subseq_model = Subseq.objects.create(
                    origin = Protein_model,
                    start = start,
                    end = end,
                    profile = None
                )\n""")
                
    log.close()

parse_data()


#Modulo.objects.create()

nbr of col {15}
nbr of rows 647
nbr of col {15}
nbr of rows 647
nbr of col {4}
nbr of rows 2797
nbr of col {4}
nbr of rows 2829
nbr of col {4}
nbr of rows 2797
nbr of col {4}
nbr of rows 2829
nbr of col {4}
nbr of rows 2797
nbr of col {4}
nbr of rows 2829
nbr of col {4}
nbr of rows 2797
nbr of col {4}
nbr of rows 2829
nbr of col {4}
nbr of rows 2797
nbr of col {4}
nbr of rows 2829
nbr of col {4}
nbr of rows 2797
nbr of col {4}
nbr of rows 2829
nbr of col {4}
nbr of rows 2797
nbr of col {4}
nbr of rows 2829
nbr of col {4}
nbr of rows 2797
nbr of col {4}
nbr of rows 2829
nbr of col {4}
nbr of rows 2797
nbr of col {4}
nbr of rows 2829
nbr of col {4}
nbr of rows 2797
nbr of col {4}
nbr of rows 2829
nbr of col {4}
nbr of rows 2797
nbr of col {4}
nbr of rows 2829
nbr of col {4}
nbr of rows 2797
nbr of col {4}
nbr of rows 2829
nbr of col {4}
nbr of rows 2797
nbr of col {4}
nbr of rows 2829
nbr of col {4}
nbr of rows 2797
nbr of col {4}
nbr of rows 2829
nbr of col {4}
nbr of rows 2797
nbr of c